## Creating types

In [82]:
from deap import base
from deap import creator
from deap import tools
import random

### Fitness

In [83]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("FintnessMulti", base.Fitness, weights=(1.0,1.0))

### Individual

### List of floats

In [84]:
from deap import base
from deap import creator
from deap import tools
import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=IND_SIZE)
toolbox.individual()

[0.1408536133903745,
 0.47997410990090683,
 0.42647841306593126,
 0.4752417511262794,
 0.43075507963588855,
 0.2335917983662128,
 0.8423490796390783,
 0.29888801885748284,
 0.27361722107599984,
 0.8716716283450413]

### Permutation

In [13]:
from deap import base
from deap import creator
from deap import tools
import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.individual()

[7, 3, 0, 6, 8, 2, 9, 5, 1, 4]

In [19]:
# tools.initIterate vs tools.initRepeat
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
print("the result of tools.initIterate")
print(toolbox.individual())

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.indices)
print("the result of tools.initRepeat")
print(toolbox.individual(1)) 

the result of tools.initIterate
[2, 5, 6, 9, 4, 0, 8, 7, 3, 1]
the result of tools.initRepeat
[[1, 6, 9, 8, 4, 2, 7, 0, 3, 5]]


### Arithmetic Expression

In [29]:
from deap import base
from deap import creator
from deap import tools
from deap import gp

import random
import operator

pset = gp.PrimitiveSet("MAIN", arity=1) # set name is called 'MAIN' and has a single variable 
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(min, 1)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.individual()

### Evolution strategy

In [38]:
from deap import base
from deap import creator
from deap import tools

import array
import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax, strategy=None)
creator.create("Strategy", array.array, typecode='d')

def initEs(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin,imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin,smax) for _ in range(size))
    return ind

IND_SIZE = 10
MIN_VALUE, MAX_VALUE = -5., 5.
MIN_STRAT, MAX_STRAT = -1., 1.

toolbox = base.Toolbox()
toolbox.register("individual", initEs, 
                creator.Individual, creator.Strategy,
                IND_SIZE, MIN_VALUE, MAX_VALUE,
                MIN_STRAT, MAX_STRAT)

toolbox.individual()

array('d', [-3.0188333498739492, -0.9183442448425652, -4.586506613508364, -1.7511170302623356, 0.983799857371018, 4.0660482018453195, 4.2051362432510135, 3.4442391167646544, 0.2854191425550878, -2.0217883855445096])

### Particle

In [73]:
# A particle is another special type of individual as it usually has a speed and generally remember its best position.

from deap import base
from deap import creator
from deap import tools

import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("particle", initParticle, creator.Particle, size=2,
                 pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.particle()

[4.104576378209526, 1.7024630623752275]

### A Funky One

In [78]:
from deap import base
from deap import creator
from deap import tools

import random

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

INT_MIN, INT_MAX = 5, 10
FLT_MIN, FLT_MAX = -0.2, 0.8
N_CYCLES = 4

toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("attr_flt", random.uniform, FLT_MIN, FLT_MAX)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_int, toolbox.attr_flt), n=N_CYCLES)
toolbox.individual()

[7,
 0.15532796403813948,
 10,
 0.723601002264451,
 6,
 -0.013718095312811374,
 10,
 0.1247750815069853]

### Population

Population are much like individuals. Instead of being initialized with attributes, they are filled with 
individuals, strategies, or particles.

In [79]:
## Bag
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.population(n=2)

## Grid
N_COL = 2
N_ROW = 2
toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=N_COL)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=N_ROW)
toolbox.population()

## Swarm
creator.create("Swarm", list, gbest=None, gbestfit=creator.FitnessMax)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)

## Demes
toolbox.register("deme", tools.initRepeat, list, toolbox.individual)

DEME_SIZE = 10, 50, 100
population = [toolbox.deme(n=i) for i in DEME_SIZE]

### Seeding a population

In [ ]:
import json

from deap import base
from deap import creator

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, filename):
    with open(filename, "r") as pop_file:
        contents = json.load(pop_file)
    return pcls(ind_init(c) for c in contents)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, "my_guess.json")

population = toolbox.population_guess()